In [ ]:
import re
import tweepy
import pandas as pd
import numpy as np
import csv
import glob
import string
from tweepy import OAuthHandler
from textblob import TextBlob
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from emot.emo_unicode import EMOJI_UNICODE

In [ ]:
# keys e tokens dalla Twitter Dev Console
consumer_key = 'HErcO0a3RQxeBc4Au44qZZknQ'
consumer_secret = 'PsC0EhWB5v9DUFVONHIMTpwRJAf211b6Ga77EFdDE9wV2Xah3S'
access_token = '2656923476-BHCZUQTee9C5KufeqFKeQC1rw91YD4O7Xx6CUxi'
access_token_secret = 'uNk9Ue5yCFmn0TxvfrrGxBOO7Z9q4QoAmn346JNS46Sby'

In [ ]:
# prova di autenticazione
try:
	# creazione dell'oggetto OAuthHandler
	auth = OAuthHandler(consumer_key, consumer_secret)
	# impostazione dell'access token e del secret
	auth.set_access_token(access_token, access_token_secret)
	# creazione dell'oggetto API tweepy per il fetch dei tweet
	api = tweepy.API(auth)
except:
	print("Error: Authentication Failed")

In [ ]:
def get_tweets(query, count = 10):
	
	# lista vuota per conservare i tweet
	tweets_list = [tweets for tweets in tweepy.Cursor(api.search, q=query, lang="en", tweet_mode='extended').items(count)]

	# scraping dei singoli tweet
	with open('tweets.csv', 'a', newline='', encoding='utf-8') as csvFile:
		column_names = ['tweet_id', 'text', 'retweet', 'likes']
		csv_writer = csv.DictWriter(csvFile, fieldnames=column_names, delimiter=',')
		csv_writer.writeheader()
		for tweet in tweets_list[::-1]:
			csv_data = {}
			csv_data['tweet_id'] = tweet.id
			csv_data['text'] = tweet.full_text
			csv_data['retweet'] = tweet.retweet_count
			csv_data['likes'] = tweet.favorite_count
			csv_writer.writerow(csv_data)

get_tweets(query= 'facebook', count = 20)

In [ ]:
tweets = []

df = pd.read_csv('tweets.csv', index_col=None, header=0)
tweets.append(df)

tweets_df = pd.concat(tweets, axis=0, ignore_index=True)

tweets_df.head()

In [ ]:
tweets_df.shape

In [ ]:
tweets_df.duplicated(subset='tweet_id').sum()

In [ ]:
tweets_df = tweets_df.drop_duplicates(subset=['tweet_id'])

In [ ]:
tweets_df.shape

In [ ]:
tweets_df.isna().any()

In [ ]:
stop_words = list(stopwords.words('english'))
alphabets = list(string.ascii_lowercase)
stop_words = stop_words + alphabets
word_list = words.words()
emojis = list(EMOJI_UNICODE.keys())

In [ ]:
def process_tweets(tweet):

    tweet = tweet.lower()

    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    tweet = re.sub(r'\@\w+|\#|\d+', '', tweet)

    tweet_tokens = word_tokenize(tweet)

    filtered_words = [w for w in tweet_tokens if w not in stop_words]
    filtered_words = [w for w in filtered_words if w not in emojis]
    filtered_words = [w for w in filtered_words if w in word_list]

    unpunctuated_words = [char for char in filtered_words if char not in string.punctuation]
    unpunctuated_words = ' '.join(unpunctuated_words)

    return "".join(unpunctuated_words)

In [ ]:
def get_adjectives(tweet):
    tweet = word_tokenize(tweet)
    tweet = [word for (word, tag) in pos_tag(tweet)
        if tag == "JJ"]
    return " ".join(tweet)

In [ ]:
tweets_df['processed tweets'] = tweets_df['text'].apply(process_tweets)

In [ ]:
tweets_df['tweets adjectives'] = tweets_df['processed tweets'].apply(get_adjectives)

In [ ]:
tweets_df.head()

In [ ]:
def lemmatize_tweets(tweet):
    tweet_tokens = word_tokenize(tweet)
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in tweet_tokens]
    return " ".join(lemma_words)

In [ ]:
tweets_df['lemmatized tweets'] = tweets_df['processed tweets'].apply(lemmatize_tweets)

In [ ]:
tweets_df.head()

In [ ]:
tweets_df.to_csv('processed_tweets.csv', encoding='utf-8-sig', index=False)

In [ ]:
def get_polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

def get_sentiment(polarity):
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
tweets_df['polarity'] = tweets_df['lemmatized tweets'].apply(get_polarity)
tweets_df['sentiment'] = tweets_df['polarity'].apply(get_sentiment)

In [ ]:
sentiment_chart = tweets_df['sentiment'].value_counts().rename_axis('sentiment').to_frame('total tweets').reset_index()
sentiment_chart